In [1]:
!pip install mlflow


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import dagshub
dagshub.init(repo_owner='Vaibha3246', repo_name='influence_mirror', mlflow=True)

Accessing as Vaibha3246

Initialized MLflow to track repo "Vaibha3246/influence_mirror"

Repository Vaibha3246/influence_mirror initialized!

In [4]:


import mlflow

mlflow.set_tracking_uri("https://dagshub.com/Vaibha3246/influence_mirror.mlflow")

with mlflow.start_run():
    mlflow.log_param("param1", 15)
    mlflow.log_metric("metric1", 0.89)

🏃 View run rogue-cow-508 at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/0/runs/90da48e787e64537905c2a9ed88d4aac
🧪 View experiment at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/0


In [5]:
import pandas as pd
df=pd.read_csv('../data/raw/youtube_bulk_raw.csv')

In [6]:
df.head()

,video_id,category,author,text,likes,published_at
0,4wZwXhoxRIA,technology,@justicebuys1,All products can be found on www.justicebuys.c...,1978,2025-01-04T19:28:08Z
1,4wZwXhoxRIA,technology,@jacklynkateisrael2568,😮😮,0,2025-09-11T09:52:55Z
2,4wZwXhoxRIA,technology,@kevinjessica6389,Bro “how to talk to woman in 6 steps” is so re...,0,2025-09-10T23:44:24Z
3,4wZwXhoxRIA,technology,@FrancescaWhite-y6y,I was gonna say does it give you the drinks fo...,0,2025-09-09T16:25:03Z
4,4wZwXhoxRIA,technology,@lucasgamingchannel-vm7pg,Anyone gonna talk abt what was o. His pc,0,2025-09-08T22:33:58Z


In [7]:
df.isnull().sum()

video_id         0
category         0
author          10
text            22
likes            0
published_at     0
dtype: int64

In [8]:
df.shape

(76693, 6)

In [9]:
df.drop(columns='author',inplace=True)

In [10]:
df[df['text'].str.strip() != ""]

,video_id,category,text,likes,published_at
0,4wZwXhoxRIA,technology,All products can be found on www.justicebuys.c...,1978,2025-01-04T19:28:08Z
1,4wZwXhoxRIA,technology,😮😮,0,2025-09-11T09:52:55Z
2,4wZwXhoxRIA,technology,Bro “how to talk to woman in 6 steps” is so re...,0,2025-09-10T23:44:24Z
3,4wZwXhoxRIA,technology,I was gonna say does it give you the drinks fo...,0,2025-09-09T16:25:03Z
4,4wZwXhoxRIA,technology,Anyone gonna talk abt what was o. His pc,0,2025-09-08T22:33:58Z
...,...,...,...,...,...
76688,4MXqlEa6M2o,education,Ye तो बेक्कूफ शिक्षा मंत्री है,27,2025-09-09T09:31:59Z
76689,4MXqlEa6M2o,education,Enke baat ka koi bharosa nahi hai.,4,2025-09-09T09:30:13Z
76690,4MXqlEa6M2o,education,15 September Tak 1 lakh + notification nikalo,39,2025-09-09T09:27:14Z
76691,4MXqlEa6M2o,education,1st & Last Education minister in Bihar,4,2025-09-09T09:24:48Z


In [11]:
df = df[df['text'].str.strip() != ""].reset_index(drop=True)
before = len(df)
df = df.drop_duplicates(subset=['video_id','text'], keep='first').reset_index(drop=True)
print("Dropped duplicates:", before - len(df))

Dropped duplicates: 19896


In [12]:
# 1. Count blanks
blank_count = (df['text'].str.strip() == "").sum()
print("Blank comments:", blank_count)

# 2. Count duplicates
dup_count = df.duplicated(subset=['video_id','text']).sum()
print("Duplicate comments:", dup_count)

# 3. Show some examples of duplicates
dupes = df[df.duplicated(subset=['video_id','text'], keep=False)]
dupes.head(10)


Blank comments: 0
Duplicate comments: 0


,video_id,category,text,likes,published_at


In [13]:
df.shape

(56797, 5)

In [14]:
!pip install emoji


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import re, html, emoji

url_re = re.compile(r'https?://\S+|www\.\S+')
mention_re = re.compile(r'@\w+')
html_tag_re = re.compile(r'<.*?>')
multispace_re = re.compile(r'\s+')

def clean_text(text, remove_emojis=False):
    s = "" if pd.isna(text) else str(text)
    s = html.unescape(s)
    s = url_re.sub(' ', s)
    s = mention_re.sub(' ', s)
    s = html_tag_re.sub(' ', s)
    if remove_emojis:
        try:
            s = emoji.replace_emoji(s, replace='')
        except Exception:
            s = s.encode('ascii', errors='ignore').decode()
    s = s.lower().strip()
    s = multispace_re.sub(' ', s)
    return s

df['text_clean'] = df['text'].apply(lambda x: clean_text(x, remove_emojis=False))
df[['text','text_clean']].head()


,text,text_clean
0,All products can be found on www.justicebuys.c...,all products can be found on 🙌🏼 since i review...
1,😮😮,😮😮
2,Bro “how to talk to woman in 6 steps” is so re...,bro “how to talk to woman in 6 steps” is so re...
3,I was gonna say does it give you the drinks fo...,i was gonna say does it give you the drinks fo...
4,Anyone gonna talk abt what was o. His pc,anyone gonna talk abt what was o. his pc


In [16]:
df[(df['text_clean'].str.strip() == '')]

,video_id,category,text,likes,published_at,text_clean
232,4wZwXhoxRIA,technology,https://youtube.com/@isha-learn?si=beDQvLlqLI4...,0,2025-05-19T09:04:49Z,
469,4wZwXhoxRIA,technology,@a2d,0,2025-05-01T07:07:46Z,
643,4wZwXhoxRIA,technology,@hopescope,0,2025-03-10T03:08:56Z,
3819,ZggspAgMsyE,gaming,https://youtu.be/gaQN_xoYPtc?sub_confirmation=1,0,2025-09-02T09:34:58Z,
3820,ZggspAgMsyE,gaming,https://www.youtube.com/live/e9_kRQPkryo?si=Ty...,0,2025-09-01T12:17:50Z,
...,...,...,...,...,...,...
52617,sUf2PtEZris,music,https://youtube.com/shorts/xGK0SThe9Kk?si=CcUF...,2,2025-05-03T06:26:12Z,
53646,sUf2PtEZris,music,https://youtube.com/shorts/2iqLFQY7xaQ?si=GUfg...,1,2025-04-24T09:03:52Z,
53670,sUf2PtEZris,music,https://youtu.be/IVNYqQWjbmM?si=DlCRtfpH-1t2Vxek,1,2025-04-24T04:59:47Z,
53713,sUf2PtEZris,music,https://youtube.com/shorts/Tbuet2MGwpk?si=w5od...,0,2025-04-23T21:32:18Z,


In [17]:
df.shape

(56797, 6)

In [18]:
df = df[~(df['text_clean'].str.strip() == '')]

In [19]:
df.head()

,video_id,category,text,likes,published_at,text_clean
0,4wZwXhoxRIA,technology,All products can be found on www.justicebuys.c...,1978,2025-01-04T19:28:08Z,all products can be found on 🙌🏼 since i review...
1,4wZwXhoxRIA,technology,😮😮,0,2025-09-11T09:52:55Z,😮😮
2,4wZwXhoxRIA,technology,Bro “how to talk to woman in 6 steps” is so re...,0,2025-09-10T23:44:24Z,bro “how to talk to woman in 6 steps” is so re...
3,4wZwXhoxRIA,technology,I was gonna say does it give you the drinks fo...,0,2025-09-09T16:25:03Z,i was gonna say does it give you the drinks fo...
4,4wZwXhoxRIA,technology,Anyone gonna talk abt what was o. His pc,0,2025-09-08T22:33:58Z,anyone gonna talk abt what was o. his pc


In [20]:
df[df['text_clean'].apply(lambda x: x.endswith(' ') or x.startswith(' '))]

,video_id,category,text,likes,published_at,text_clean


In [21]:
# Remove trailing and leading whitespaces from the 'clean_comment' column
df['text_clean'] = df['text_clean'].str.strip()

In [22]:
df.shape

(56252, 6)

In [23]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

# Function to classify sentiment
def get_sentiment(text):
    score = analyzer.polarity_scores(str(text))['compound']
    if score >= 0.05:
        return "positive"
    elif score <= -0.05:
        return "negative"
    else:
        return "neutral"

# Apply sentiment on cleaned text
df['sentiment'] = df['text_clean'].apply(get_sentiment)

# Check results
print(df[['text_clean', 'sentiment']].head(10))
print("\nSentiment distribution:")
print(df['sentiment'].value_counts())


                                          text_clean sentiment
0  all products can be found on 🙌🏼 since i review...  positive
1                                                 😮😮   neutral
2  bro “how to talk to woman in 6 steps” is so re...   neutral
3  i was gonna say does it give you the drinks fo...  positive
4           anyone gonna talk abt what was o. his pc   neutral
5  how is everyone not talking about his search?!...  positive
6               when tech goes from wow to how?! 😳👉🧠  positive
7                        this is why l love this guy  positive
8                                    i have number 8  positive
9  imagine using that flashlight 🔦 in the middle ...  positive

Sentiment distribution:
sentiment
positive    34606
neutral     16634
negative     5012
Name: count, dtype: int64


In [25]:
df.sample(10)

,video_id,category,text,likes,published_at,text_clean,sentiment
20837,sUf2PtEZris,music,"29 july 2025 ko kon kon dekh rahe ho,,??",4,2025-07-28T20:01:18Z,"29 july 2025 ko kon kon dekh rahe ho,,??",neutral
34185,sUf2PtEZris,music,How many of you listen at least one time per d...,3,2025-06-25T14:48:55Z,how many of you listen at least one time per d...,positive
7674,iFIbm4Xqjwo,movies,Bro is swinging like a complete 🐱,0,2025-07-26T17:02:14Z,bro is swinging like a complete 🐱,positive
25968,sUf2PtEZris,music,I am here after laughter chef 😂,3,2025-07-14T16:19:12Z,i am here after laughter chef 😂,positive
49658,sUf2PtEZris,music,Viral in Nepal 🔥,0,2025-05-21T12:08:00Z,viral in nepal 🔥,negative
53179,sUf2PtEZris,music,Global level video quality and concept and son...,3,2025-04-27T06:57:51Z,global level video quality and concept and son...,positive
21503,sUf2PtEZris,music,27 July ko kon kon sun rha h?,0,2025-07-26T16:54:22Z,27 july ko kon kon sun rha h?,neutral
3034,vwTL0SLEl1M,technology,"i'm still using HDMI, and my monitor only supp...",0,2025-08-26T05:34:47Z,"i'm still using hdmi, and my monitor only supp...",positive
1937,4wZwXhoxRIA,technology,nice try diddy,1,2025-01-05T07:57:16Z,nice try diddy,positive
55214,o-YjuXcZ9l8,education,Please sir 🙏,0,2025-09-11T14:19:16Z,please sir 🙏,positive


In [26]:
df['text_clean']=df['text_clean'].str.replace('\n', ' ', regex=True)


In [27]:
df['sentiment_numeric'] = df['sentiment'].replace({
    'positive': 1,
    'negative': -1,
    'neutral': 0
})


C:\Users\vaibh\AppData\Local\Temp\ipykernel_6740\1706387719.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment_numeric'] = df['sentiment'].replace({


In [28]:
df['word_count']=df['text_clean'].apply(lambda x: len(x.split()))

In [29]:
df.sample()

,video_id,category,text,likes,published_at,text_clean,sentiment,sentiment_numeric,word_count
2786,vwTL0SLEl1M,technology,Please convince my dad that he will let me buy...,0,2025-08-26T13:26:46Z,please convince my dad that he will let me buy...,positive,1,27


In [30]:
# Word frequency → shows most common terms used (good for making word clouds,
# stopwords analysis).

from collections import Counter

# Combine all comments into one large string
all_text = ' '.join(df['text_clean'])

# Split into words and count
word_frequency = Counter(all_text.split())

# Convert into DataFrame
word_frequency_df = pd.DataFrame(word_frequency.items(), columns=['word', 'frequency']).sort_values(by='frequency', ascending=False)

print(word_frequency_df.head(20))  # top 20 words


         word  frequency
15159    song       8056
9424      kon       7176
40        the       6545
8           i       4614
30         is       4565
24         to       4010
8814      hai       3978
14          a       3748
69       this       3602
125       and       3502
39        you       3085
8890       ko       2820
44     anyone       2695
27         in       2640
9323     dekh       2525
757         ❤       2493
85       like       2484
37         it       2443
8987     raha       2439
82         of       2420


In [31]:
!pip install nltk


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
df = df[(df['word_count'] >= 2) & (df['word_count'] <= 100)]


In [33]:
from nltk.corpus import stopwords

# Download NLTK stopwords if not already downloaded
import nltk
nltk.download('stopwords')

# Define the list of English stopwords
stop_words = set(stopwords.words('english'))

# Create a new column 'num_stop_words' by counting the number of stopwords in each comment
df['num_stop_words'] = df['text_clean'].apply(lambda x: len([word for word in x.split() if word in stop_words]))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaibh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
# Create a new column 'num_punctuation_chars' to count punctuation characters in each comment
df['num_punctuation_chars'] = df['text_clean'].apply(
    lambda x: sum([1 for char in x if char in '.,!?;:"\'()[]{}-'])
)

df.sample(5)

,video_id,category,text,likes,published_at,text_clean,sentiment,sentiment_numeric,word_count,num_stop_words,num_punctuation_chars
8617,GJZAcj4P99g,music,Happy Onam 🎉❤❤❤🎉🎉,0,2025-09-05T16:44:10Z,happy onam 🎉❤❤❤🎉🎉,positive,1,3,0,0
51786,sUf2PtEZris,music,Ek number lots of love from goa,4,2025-05-07T22:23:40Z,ek number lots of love from goa,positive,1,7,2,0
15650,sUf2PtEZris,music,The right and left girl is like a ghost😂😂😂😂 bu...,0,2025-08-18T09:40:05Z,the right and left girl is like a ghost😂😂😂😂 bu...,positive,1,20,9,0
3105,vwTL0SLEl1M,technology,will it be a consumer product tho?,0,2025-08-26T03:05:25Z,will it be a consumer product tho?,neutral,0,7,4,1
53362,sUf2PtEZris,music,Lavkar ch Trending la 🥰🥰,1,2025-04-26T04:43:29Z,lavkar ch trending la 🥰🥰,positive,1,5,0,0


In [35]:
df.sample(5)

,video_id,category,text,likes,published_at,text_clean,sentiment,sentiment_numeric,word_count,num_stop_words,num_punctuation_chars
46251,sUf2PtEZris,music,Never imagined such song... Wow❤❤❤❤so addictive,1,2025-06-01T17:02:33Z,never imagined such song... wow❤❤❤❤so addictive,positive,1,6,1,3
20194,sUf2PtEZris,music,Best. Song. Sanju. Bhai,0,2025-07-31T08:15:10Z,best. song. sanju. bhai,positive,1,4,0,3
20187,sUf2PtEZris,music,31 ko kon kon dek rehai ho 😅😅😅😂,0,2025-07-31T09:05:56Z,31 ko kon kon dek rehai ho 😅😅😅😂,positive,1,8,0,0
20829,sUf2PtEZris,music,Sindhi + English,0,2025-07-28T23:06:30Z,sindhi + english,neutral,0,3,0,0
27786,sUf2PtEZris,music,So nice song!!❤❤❤,0,2025-07-10T09:16:39Z,so nice song!!❤❤❤,positive,1,3,1,2


In [36]:
df['num_chars'] = df['text_clean'].apply(len)

df.head()

,video_id,category,text,likes,published_at,text_clean,sentiment,sentiment_numeric,word_count,num_stop_words,num_punctuation_chars,num_chars
0,4wZwXhoxRIA,technology,All products can be found on www.justicebuys.c...,1978,2025-01-04T19:28:08Z,all products can be found on 🙌🏼 since i review...,positive,1,24,9,1,116
2,4wZwXhoxRIA,technology,Bro “how to talk to woman in 6 steps” is so re...,0,2025-09-10T23:44:24Z,bro “how to talk to woman in 6 steps” is so re...,neutral,0,12,5,0,53
3,4wZwXhoxRIA,technology,I was gonna say does it give you the drinks fo...,0,2025-09-09T16:25:03Z,i was gonna say does it give you the drinks fo...,positive,1,12,7,1,54
4,4wZwXhoxRIA,technology,Anyone gonna talk abt what was o. His pc,0,2025-09-08T22:33:58Z,anyone gonna talk abt what was o. his pc,neutral,0,9,3,1,40
5,4wZwXhoxRIA,technology,HOW IS EVERYONE NOT TALKING ABOUT HIS SEARCH?!...,0,2025-09-08T12:17:37Z,how is everyone not talking about his search?!...,positive,1,15,6,4,85


In [37]:
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

# Define the lemmatizer
lemmatizer = WordNetLemmatizer()

# Apply lemmatization to the 'clean_comment_no_stopwords' column
df['text_clean'] = df['text_clean'].apply(
    lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()])
)

df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vaibh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,video_id,category,text,likes,published_at,text_clean,sentiment,sentiment_numeric,word_count,num_stop_words,num_punctuation_chars,num_chars
0,4wZwXhoxRIA,technology,All products can be found on www.justicebuys.c...,1978,2025-01-04T19:28:08Z,all product can be found on 🙌🏼 since i review ...,positive,1,24,9,1,116
2,4wZwXhoxRIA,technology,Bro “how to talk to woman in 6 steps” is so re...,0,2025-09-10T23:44:24Z,bro “how to talk to woman in 6 steps” is so re...,neutral,0,12,5,0,53
3,4wZwXhoxRIA,technology,I was gonna say does it give you the drinks fo...,0,2025-09-09T16:25:03Z,i wa gonna say doe it give you the drink for f...,positive,1,12,7,1,54
4,4wZwXhoxRIA,technology,Anyone gonna talk abt what was o. His pc,0,2025-09-08T22:33:58Z,anyone gonna talk abt what wa o. his pc,neutral,0,9,3,1,40
5,4wZwXhoxRIA,technology,HOW IS EVERYONE NOT TALKING ABOUT HIS SEARCH?!...,0,2025-09-08T12:17:37Z,how is everyone not talking about his search?!...,positive,1,15,6,4,85


In [38]:
df['published_at'] = pd.to_datetime(df['published_at'])
df['hour'] = df['published_at'].dt.hour
df['weekday'] = df['published_at'].dt.weekday
df['month'] = df['published_at'].dt.month


In [39]:
from sklearn.preprocessing import OneHotEncoder

# ✅ Handle both sklearn versions
try:
    ohe = OneHotEncoder(sparse_output=False, drop="first")  # new versions
except TypeError:
    ohe = OneHotEncoder(sparse=False, drop="first")  # older versions

category_encoded = ohe.fit_transform(df[['category']])

# Convert back to DataFrame with category names
category_df = pd.DataFrame(category_encoded, columns=ohe.get_feature_names_out(['category']))

# Merge into original DataFrame
df = pd.concat([df.reset_index(drop=True), category_df.reset_index(drop=True)], axis=1)

df.head()


,video_id,category,text,likes,published_at,text_clean,sentiment,sentiment_numeric,word_count,num_stop_words,num_punctuation_chars,num_chars,hour,weekday,month,category_gaming,category_movies,category_music,category_technology
0,4wZwXhoxRIA,technology,All products can be found on www.justicebuys.c...,1978,2025-01-04 19:28:08+00:00,all product can be found on 🙌🏼 since i review ...,positive,1,24,9,1,116,19,5,1,0.0,0.0,0.0,1.0
1,4wZwXhoxRIA,technology,Bro “how to talk to woman in 6 steps” is so re...,0,2025-09-10 23:44:24+00:00,bro “how to talk to woman in 6 steps” is so re...,neutral,0,12,5,0,53,23,2,9,0.0,0.0,0.0,1.0
2,4wZwXhoxRIA,technology,I was gonna say does it give you the drinks fo...,0,2025-09-09 16:25:03+00:00,i wa gonna say doe it give you the drink for f...,positive,1,12,7,1,54,16,1,9,0.0,0.0,0.0,1.0
3,4wZwXhoxRIA,technology,Anyone gonna talk abt what was o. His pc,0,2025-09-08 22:33:58+00:00,anyone gonna talk abt what wa o. his pc,neutral,0,9,3,1,40,22,0,9,0.0,0.0,0.0,1.0
4,4wZwXhoxRIA,technology,HOW IS EVERYONE NOT TALKING ABOUT HIS SEARCH?!...,0,2025-09-08 12:17:37+00:00,how is everyone not talking about his search?!...,positive,1,15,6,4,85,12,0,9,0.0,0.0,0.0,1.0


In [ ]:
#  Add VADER Sentiment Scores
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

df[['sent_neg', 'sent_neu', 'sent_pos', 'sent_compound']] = df['text_clean'].apply(
    lambda x: pd.Series(sia.polarity_scores(x))
)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\vaibh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [41]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [42]:
# Drop non-numeric columns
X = df.drop(columns=['video_id','category','text','text_clean','sentiment','published_at','sentiment_numeric'])
y = df['sentiment_numeric']


In [43]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [49]:
# setup mlflow tracking server

mlflow.set_tracking_uri("https://dagshub.com/Vaibha3246/influence_mirror.mlflow")

In [ ]:
# Set or create an experiment
mlflow.set_experiment("RF Baseline") 

2025/09/26 11:27:44 INFO mlflow.tracking.fluent: Experiment with name 'RF Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/008c5a930fca452a8fa9d2ef4f21e105', creation_time=1758866263515, experiment_id='0', last_update_time=1758866263515, lifecycle_stage='active', name='RF Baseline', tags={}>

In [51]:
# ✅ Step 0: Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn



# Split data into train/test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ✅ Step 2: Define and train  a Random Forest baseline model using MLflow
with mlflow.start_run() as run:
    # --- MLflow Run Tags ---
    mlflow.set_tag("mlflow.runName", "RandomForest_Baseline_TrainTestSplit")
    mlflow.set_tag("experiment_type", "baseline")
    mlflow.set_tag("model_type", "RandomForestClassifier")
    mlflow.set_tag(
        "description",
        "Baseline RandomForest model for sentiment analysis  with a simple train-test split",
    )

    # --- Model Parameters ---
    n_estimators = 200
    max_depth = 15
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)

    # --- Train Model ---
    model = RandomForestClassifier(
        n_estimators=n_estimators, max_depth=max_depth, random_state=42
    )
    model.fit(X_train, y_train)

    # --- Predictions ---
    y_pred = model.predict(X_test)

    # --- Metrics ---
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    classification_rep = classification_report(y_test, y_pred, output_dict=True)
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):  # Only log precision, recall, f1-score
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)

    # --- Confusion Matrix ---
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")

    # Save and log confusion matrix plot
    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")
    plt.close()

    # --- Log Model ---
    mlflow.sklearn.log_model(model, "random_forest_model")

    # --- Optionally Log Dataset (if small) ---
    # Example: if you have your DataFrame as df
    df.to_csv("dataset.csv", index=False)
    mlflow.log_artifact("dataset.csv")

# ✅ Print Final Accuracy
print(f"✅ Accuracy: {accuracy:.4f}")


2025/09/26 11:29:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run RandomForest_Baseline_TrainTestSplit at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/0/runs/0900e0a7b0484667a31580faf09d4e24
🧪 View experiment at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/0
✅ Accuracy: 0.7269


In [47]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.79      0.63      0.70       938
           0       0.65      0.42      0.51      2875
           1       0.74      0.89      0.81      6094

    accuracy                           0.73      9907
   macro avg       0.73      0.65      0.67      9907
weighted avg       0.72      0.73      0.71      9907



In [48]:
df.to_csv('reddit_preprocessing.csv', index=False)